In [ ]:
import spotipy
from spotipy import util
from pprint import pprint
import random
import string
import pandas as pd
import numpy as np
import json
import os
import csv

username = 'ernyjyqoecerasugt8dpmhbmo'
scope = 'playlist-modify-public'
my_client_id = '83c842b2f4d447b6b2a19df91847487d'
my_client_secret = '8bd6101237114d9eb851e2d3183affce'

In [ ]:
spotify_token = util.prompt_for_user_token(username, 
                                           scope, 
                                           client_id=my_client_id, 
                                           client_secret=my_client_secret,
                                           redirect_uri='http://example.com/callback/')

In [ ]:
sp = spotipy.Spotify(auth=spotify_token)


In [ ]:
#inputCSV = os.path.join('.', 'Resources', 'test_Hot_100BBCharts_time.csv')
#inputCSV = os.path.join('.', 'Resources','HotList_Splits_Dir', 'xaa.csv')
#inputCSV = os.path.join('.', 'Resources','HotList_Splits_Dir', 'xab.csv')
#inputCSV = os.path.join('.', 'Resources','HotList_Splits_Dir', 'xac.csv')
#inputCSV = os.path.join('.', 'Resources','HotList_Splits_Dir', 'xad.csv')
#inputCSV = os.path.join('.', 'Resources','HotList_Splits_Dir', 'xae.csv')
#inputCSV = os.path.join('.', 'Resources','HotList_Splits_Dir', 'xaf.csv')
#inputCSV = os.path.join('.', 'Resources','HotList_Splits_Dir', 'xag.csv')
#inputCSV = os.path.join('.', 'Resources','HotList_Splits_Dir', 'xah.csv')
#inputCSV = os.path.join('.', 'Resources','HotList_Splits_Dir', 'xai.csv')
inputCSV = os.path.join('.', 'Resources','HotList_Splits_Dir', 'xaj.csv')
#inputCSV = os.path.join('.', 'Resources', 'Hot_100BBCharts_time.csv')

billboardChartsDF = pd.read_csv(inputCSV, header=0)
bbcGroupTA = billboardChartsDF.groupby(['Title','Artist']).groups

billboardChartsDF['Title'].value_counts()

In [ ]:
#barrowed from audio_features.py and artist_discography.py

outputCSVa = os.path.join('.', 'Resources', 'HotList_SongFeat_Dir','xaj_SongFeats_Successful.csv')
outputCSVb = os.path.join('.', 'Problems', 'HotList_SongFeat_Dir','xaj_MissingSongData.csv')
outputCSVc = os.path.join('.', 'Problems', 'HotList_SongFeat_Dir','xaj_NeedNext_url_data.csv')
outputCSVd = os.path.join('.', 'Problems', 'HotList_SongFeat_Dir','xaj_Invalid_URI_data.csv')

with open(outputCSVa, "w", newline="") as outCSVFileA, open(outputCSVb, "w", newline="") as outCSVFileB, open(outputCSVc, "w", newline="") as outCSVFileC, open(outputCSVd, "w", newline="") as outCSVFileD:
    #headersA =['SongName','SongArtist','SongPopularity','ReleaseDate','Danceability','Energy','Key','Loudness','Mode','Speechiness','Acousticness','Instrumentalness','Liveness','Valence','Tempo','SongType','SongfeatID','SonguURI','Track_href','Analysis_URL','Duration_ms','TimeSignature','Genres']
    headersA =['SongName','SongArtist','SongPopularity','ReleaseDate','Danceability','Energy','Key','Loudness','Mode','Speechiness','Acousticness','Instrumentalness','Liveness','Valence','Tempo','SongType','SongfeatID','SonguURI','Track_href','Analysis_URL','Duration_ms','TimeSignature']
    writerA = csv.DictWriter(outCSVFileA,fieldnames=headersA,extrasaction='ignore')
    writerA.writeheader()
    headersB = ['MissingSong', 'MissingArtist']
    writerB = csv.DictWriter(outCSVFileB,fieldnames=headersB,extrasaction='ignore')
    writerB.writeheader()
    headersC = ['SongName','SongArtist','Next_URL']
    writerC = csv.DictWriter(outCSVFileC,fieldnames=headersC,extrasaction='ignore')
    writerC.writeheader()
    headersD = ['Invalid_URI_Song','Invalid_URI_Artist','Invalid_URI']
    writerD = csv.DictWriter(outCSVFileD,fieldnames=headersD,extrasaction='ignore')
    writerD.writeheader()
    
    scount = 0
    
    for song, artist_name in bbcGroupTA:
        #artist_name = 'Bobby Darin'
        #song = 'Splish Splash'
        scount += 1
        
        results = sp.search(q=artist_name, limit=50)
        
        tids = []
        testData = {}
        missingData = {}
        nextURL = {}
        invalidURI = {}
        
        print(f"\nGOAL***Song: {song} by {artist_name}... counter {scount}\n")
        #print(json.dumps(results, indent = 4))
        
        if not results["tracks"]["total"] == 0:
        
            for i, list in enumerate(results['tracks']['items']):
        
                if song == list['name']:
                    testData.update(
                        SongName = song,
                        SongArtist = artist_name,
                        SongPopularity = results["tracks"]["items"][i]["popularity"],
                        ReleaseDate = results["tracks"]["items"][i]["album"]["release_date"]
                         )
                    #print(f" {i} {list['name']} by {artist_name} Successful on to features\n")
                    tids.append(list['uri'])
                    break
                    
                    #need loser matching but not too loose... fuzzymh?
 
                elif i == 49:
                    nextURL.update(
                        SongName = song,
                        SongArtist = artist_name,
                        Next_URL = results["tracks"]["next"]
                        )
                    #print(f"Need Next URL Results: {nextURL}\n")
                        
            if len(tids) > 0:   
                features = sp.audio_features(tids)
            
                if not features:
                
                    for feature in features:
                        #print(json.dumps(feature, indent=4))
                
                        testData.update(
                            Danceability = feature["danceability"],
                            Energy = feature["energy"],
                            Key = feature["key"],
                            Loudness = feature["loudness"],
                            Mode = feature["mode"],
                            Speechiness = feature["speechiness"],
                            Acousticness = feature["acousticness"],
                            Instrumentalness = feature["instrumentalness"],
                            Liveness = feature["liveness"],
                            Valence = feature["valence"],
                            Tempo = feature["tempo"],
                            SongType = feature["type"],
                            SongfeatID = feature["id"],
                            SonguURI = feature["uri"],
                            Track_href = feature["track_href"],
                            Analysis_URL = feature["analysis_url"],
                            Duration_ms = feature["duration_ms"],
                            TimeSignature = feature["time_signature"]
                            )
                        #print(f"\n" + "="*25 + " Successful feats " + "="*25 + "\n" )
                        
                else:        
                    invalidURI.update(
                                     Invalid_URI_Song = song,
                                     Invalid_URI_Artist = artist_name,
                                     Invalid_URI = tids[0] 
                                     )
                                    
                #artistOBJ = sp.search(q='artist:' + artist_name, type='artist', limit=20)
                
                #print(json.dumps(artistOBJ, indent = 4))
                
                #if not artistOBJ:
                #    testData.update(
                #                   Genres = "NULL"
                #                   )
                #else:
                #    if len(artistOBJ['artists']['items']) > 0:
                #        testData.update(
                #                       Genres = artistOBJ['artists']['items'][3]
                #                       )
            
        else:
            missingData.update(
                MissingSong = song,
                MissingArtist = artist_name
                )      
            #print(f"\n" + "="*25 + " Missing Song {song} and Artist {artist_name} INFO " + "="*25 + "\n" )
        
        if len(testData) > 0:
            writerA.writerow(testData)
            
        elif len(missingData) > 0:
            writerB.writerow(missingData)
            
        elif len(nextURL) > 0:
            writerC.writerow(nextURL)
                        
        elif len(invalidURI) > 0:
            writerD.writerow(nextURL)
            
print(f"\nThat's all....\n")            
            
            